In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)
def j(n,m):return L(xjNew(n,m)/x)
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    return polynomialCoefficient(dg,polynomial)
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

In [2]:
print(j(5,3));print("");print(J(5,3))

x^-1 + 744 + 196884*x + 21493760*x^2 + 864299970*x^3 + 20245856256*x^4

x^-1 + 31/72 + 1823/27648*x + 10495/2519424*x^2 + 1778395/18345885696*x^3 + 45767/34828517376*x^4 + 41650330075/3327916660110655488*x^5 + O(x^6)


In [14]:
poly=xjNew(5,3);
print(poly)
print(poly.degree())
print list(poly)

20245856256*x^5 + 864299970*x^4 + 21493760*x^3 + 196884*x^2 + 744*x + 1
5
[1, 744, 196884, 21493760, 864299970, 20245856256]


In [3]:
print j(5,3)

x^-1 + 744 + 196884*x + 21493760*x^2 + 864299970*x^3 + 20245856256*x^4


In [24]:
data=[]
for m in [3..12]:
    if 2 in divisors(m):
        print(m)
    data=data+[[m,j(5,m)]]
import pickle
wfile = open('/Users/barrybrent/8mar21no3.txt','a')
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

4
6
8
10
12


In [25]:
import pickle
rfile = open('/Users/barrybrent/8mar21no3.txt','r')
data=[]
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print(s)

[[3, 20245856256*x^4 + 864299970*x^3 + 21493760*x^2 + 196884*x + 1/x + 744], [4, 11218454577152*x^4 + 81264771072*x^3 + 394264576*x^2 + 1119232*x + 1/x + 1664], [5, 611783148748800*x^4 + 1669770821250*x^3 + 3316992000*x^2 + 4287700*x + 1/x + 3160], [6, 12765329998479360*x^4 + 17546820452352*x^3 + 18186502144*x^2 + 12828672*x + 1/x + 5376], [7, 763277744265838592/5*x^4 + 122421510269730*x^3 + 75420047360*x^2 + 32384884*x + 1/x + 8456], [8, 18847818977487355904/15*x^4 + 644070915440640*x^3 + 769403125760/3*x^2 + 72208384*x + 1/x + 12544], [9, 39426163455019106304/5*x^4 + 2751723239454882*x^3 + 751427905536*x^2 + 146452212*x + 1/x + 17784], [10, 120667058077486284800/3*x^4 + 10012474736640000*x^3 + 5880414208000/3*x^2 + 275660800*x + 1/x + 24320], [11, 870684224968695627776/5*x^4 + 32056430053661442*x^3 + 4657814779904*x^2 + 488459092*x + 1/x + 32296], [12, 3298990680702153916416/5*x^4 + 92450199949148160*x^3 + 10252263096320*x^2 + 823440384*x + 1/x + 41856]]


In [2]:
data=[]
for m in [3..500]:
    if 10 in divisors(m):
        print(m)
    data=data+[[m,j(120,m)]]
import pickle
wfile = open('/Users/barrybrent/8mar21no6.txt','a')
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500


In [3]:
poly=x+2*x^3
polynomialCoefficient(0,poly)

0

In [2]:
import pickle
rfile = open('/Users/barrybrent/8mar21no6.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
polydata=[]
import time
wfile = open('/Users/barrybrent/10mar21no7.txt','w')
enoughfile=[]
for qpower in [0..5]:
    print(qpower)
    start = time.time()
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=x*s[k][1] 
# times x because, to use polynomialCoefficient, I need a polynomial; not a Laurent series
        cf=polynomialCoefficient(qpower,poly)
        data=data+[[m,cf]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[qpower-1,rl]] 
    finish = time.time()
    print(finish-start)
# -1 because I want to record the coefficients of the qpowers of the original Laurent series
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

0
4.07489395142
1
3.52210116386
2
3.54731011391
3
3.52560997009
4
3.55610609055
5
3.49046397209


In [2]:
import pickle
rfile = open('/Users/barrybrent/10mar21no7.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [0..len(s)-1]:
    print(s[n][0])
    print(s[n][1])
    print("---------------------------------------------------------------------------------")

-1
1
---------------------------------------------------------------------------------
0
24*x^3 + 32*x
---------------------------------------------------------------------------------
1
276*x^6 - 32*x^4 - 192*x^2
---------------------------------------------------------------------------------
2
2048*x^9 - 237568/27*x^7 + 32768/27*x^5 + 131072/27*x^3
---------------------------------------------------------------------------------
3
11202*x^12 - 122272*x^10 + 332480*x^8 - 51712*x^6 - 155136*x^4
---------------------------------------------------------------------------------
4
49152*x^15 - 1072627712/1125*x^13 + 4173856768/675*x^11 - 45736787968/3375*x^9 + 1564475392/675*x^7 + 6257901568/1125*x^5
---------------------------------------------------------------------------------


In [4]:
# above interpolating polynomials look correct, so do the full routine:
import pickle
rfile = open('/Users/barrybrent/8mar21no6.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
polydata=[]
import time
wfile = open('/Users/barrybrent/10mar21no8.txt','w')
enoughfile=[]
for qpower in [0..120]:
    start = time.time()
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=x*s[k][1] 
# times x because, to use polynomialCoefficient, I need a polynomial; not a Laurent series
        cf=polynomialCoefficient(qpower,poly)
        data=data+[[m,cf]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[qpower-1,rl]] 
    finish = time.time()
    print([qpower,finish-start])
# -1 because I want to record the coefficients of the qpowers of the original Laurent series
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

[0, 3.476809024810791]
[1, 3.5383589267730713]
[2, 3.4991610050201416]
[3, 3.594343900680542]
[4, 3.532088041305542]
[5, 3.5358738899230957]
[6, 3.5763092041015625]
[7, 3.5106911659240723]
[8, 3.568726062774658]
[9, 3.5482139587402344]
[10, 3.553571939468384]
[11, 3.594465970993042]
[12, 3.520862102508545]
[13, 3.532066822052002]
[14, 3.661604166030884]
[15, 3.5511369705200195]
[16, 3.592189073562622]
[17, 3.567767858505249]
[18, 3.59506893157959]
[19, 3.631237030029297]
[20, 3.5660011768341064]
[21, 3.608276844024658]
[22, 3.559959888458252]
[23, 3.584864854812622]
[24, 3.6208600997924805]
[25, 3.5829501152038574]
[26, 3.615128993988037]
[27, 3.6606009006500244]
[28, 3.621027946472168]
[29, 3.6795859336853027]
[30, 3.7123470306396484]
[31, 3.688579797744751]
[32, 3.6764211654663086]
[33, 3.658989191055298]
[34, 3.6817078590393066]
[35, 3.6548259258270264]
[36, 3.682342767715454]
[37, 3.721776008605957]
[38, 3.702333927154541]
[39, 3.7165682315826416]
[40, 3.744623899459839]
[41, 3.689

In [5]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [0..4]:
    print(s[n][0])
    f=s[n][1]
    print(f);print(f.factor())
    print("---------------------------------------------------------------------------------")

-1
1
1
---------------------------------------------------------------------------------
0
24*x^3 + 32*x
8*(3*x^2 + 4)*x
---------------------------------------------------------------------------------
1
276*x^6 - 32*x^4 - 192*x^2
4*(69*x^4 - 8*x^2 - 48)*x^2
---------------------------------------------------------------------------------
2
2048*x^9 - 237568/27*x^7 + 32768/27*x^5 + 131072/27*x^3
2048/27*(27*x^4 - 8*x^2 - 16)*(x + 2)*(x - 2)*x^3
---------------------------------------------------------------------------------
3
11202*x^12 - 122272*x^10 + 332480*x^8 - 51712*x^6 - 155136*x^4
2*(5601*x^6 - 38732*x^4 + 11312*x^2 + 19392)*(x + 2)*(x - 2)*x^4
---------------------------------------------------------------------------------


In [2]:
import pickle
rsfile = open('/Users/barrybrent/10mar21no8.txt','r')
# SageMath file
rs=pickle.load(rsfile)
rsfile.close()
s=stripQuotationMarks(rs)
rtfile = open('/Users/barrybrent/run20nov20no6','r')
# Mathematica file
L=rtfile.readlines()
rtfile.close()
data=[]
for n in [1..98]:
    f=s[n][1]
    g = D(L[n-1])
    data=data+[expand(D(f-g))]
print[data]

# This shows that the SageMath file and the Mathematica file essentially agree.



[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
